In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 20 #10:20%, 20: 40%, 30:60%
train_image_s2 = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class!=2))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
# sum(Train_participant_class)


## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

## Speaker 6 Test dataset 

In [15]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [16]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [17]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [18]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [19]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [20]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [21]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [22]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [23]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [24]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_training_Data_Size.csv')
# Perfomance
Perfomance                                                        

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,10,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,10,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,10,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,10,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,10,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Group,40%,54.661631,0.9871,0.9724,0.991,0.9730,0.9722,0.9167,0.9913,0.9826,0.9817,0.9908,1.00,1.00,0.68,0.424,0.2475
76,Group,40%,55.826749,0.9890,0.9724,0.991,0.9640,0.9907,0.9352,0.9913,0.9826,0.9725,0.9817,1.00,1.00,0.71,0.448,0.3465
77,Group,40%,55.918525,0.9945,0.9761,0.991,0.9640,1.0000,0.9352,0.9913,0.9913,0.9908,0.9908,1.00,1.00,0.70,0.440,0.3267
78,Group,40%,55.305840,0.9926,0.9761,0.982,0.9730,1.0000,0.9259,0.9913,0.9913,0.9908,0.9908,1.00,1.00,0.70,0.480,0.2673


# Create model 

In [25]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [26]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Group",'Size':'20&s2_30','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_mix_20&s2_30_0608.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20&s2_30_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20&s2_30_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
18/18 [==============================] - 5s 268ms/step - loss: 3.3001 - participant_output_loss: 1.2902 - command_output_loss: 2.0061 - command_output_1_loss: 4.3460e-04 - participant_output_1_loss: 0.0034 - participant_output_accuracy: 0.5055 - command_output_accuracy: 0.4618 - command_output_1_accuracy: 0.0036 - participant_output_1_accuracy: 0.0836 - val_loss: 2.7509 - val_participant_output_loss: 1.0014 - val_command_output_loss: 1.7492 - val_command_output_1_loss: 1.1479e-04 - val_participant_output_1_loss: 1.6156e-04 - val_participant_output_accuracy: 0.7182 - val_command_output_accuracy: 0.7772 - val_command_output_1_accuracy: 0.0129 - val_participant_output_1_accuracy: 0.1271
Epoch 2/10
18/18 [==============================] - 3s 155ms/step - loss: 2.4326 - participant_output_loss: 0.7996 - command_output_loss: 1.6328 - command_output_1_loss: 8.4837e-05 - participant_output_1_loss: 9.4877e-05 - participant_output_accuracy: 0.8364 - command_output_accuracy: 0.8445 - c

Epoch 3/10
18/18 [==============================] - 2s 120ms/step - loss: 0.2556 - participant_output_loss: 0.0526 - command_output_loss: 0.2030 - command_output_1_loss: 2.5750e-05 - participant_output_1_loss: 1.7410e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991 - command_output_1_accuracy: 0.1991 - participant_output_1_accuracy: 0.2145 - val_loss: 0.3572 - val_participant_output_loss: 0.1079 - val_command_output_loss: 0.2493 - val_command_output_1_loss: 2.6612e-05 - val_participant_output_1_loss: 2.3387e-05 - val_participant_output_accuracy: 0.9779 - val_command_output_accuracy: 0.9816 - val_command_output_1_accuracy: 0.1363 - val_participant_output_1_accuracy: 0.1694
Epoch 4/10
18/18 [==============================] - 3s 151ms/step - loss: 0.2184 - participant_output_loss: 0.0462 - command_output_loss: 0.1721 - command_output_1_loss: 2.4805e-05 - participant_output_1_loss: 1.5996e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991

Epoch 5/10
18/18 [==============================] - 3s 184ms/step - loss: 1.9269 - participant_output_loss: 0.4504 - command_output_loss: 1.4764 - command_output_1_loss: 4.3081e-05 - participant_output_1_loss: 3.1877e-05 - participant_output_accuracy: 0.9691 - command_output_accuracy: 0.9555 - command_output_1_accuracy: 0.1891 - participant_output_1_accuracy: 0.2400 - val_loss: 1.8915 - val_participant_output_loss: 0.4727 - val_command_output_loss: 1.4187 - val_command_output_1_loss: 2.5278e-05 - val_participant_output_1_loss: 3.5143e-05 - val_participant_output_accuracy: 0.9595 - val_command_output_accuracy: 0.9503 - val_command_output_1_accuracy: 0.1750 - val_participant_output_1_accuracy: 0.1344
Epoch 6/10
18/18 [==============================] - 3s 177ms/step - loss: 1.7163 - participant_output_loss: 0.3840 - command_output_loss: 1.3322 - command_output_1_loss: 5.6315e-05 - participant_output_1_loss: 2.8674e-05 - participant_output_accuracy: 0.9791 - command_output_accuracy: 0.9755

Epoch 7/10
18/18 [==============================] - 3s 153ms/step - loss: 0.3876 - participant_output_loss: 0.0644 - command_output_loss: 0.3232 - command_output_1_loss: 2.5561e-05 - participant_output_1_loss: 1.6349e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991 - command_output_1_accuracy: 0.3209 - participant_output_1_accuracy: 0.2409 - val_loss: 0.4869 - val_participant_output_loss: 0.1231 - val_command_output_loss: 0.3638 - val_command_output_1_loss: 1.6638e-05 - val_participant_output_1_loss: 2.3250e-05 - val_participant_output_accuracy: 0.9834 - val_command_output_accuracy: 0.9853 - val_command_output_1_accuracy: 0.5562 - val_participant_output_1_accuracy: 0.2376
Epoch 8/10
18/18 [==============================] - 3s 159ms/step - loss: 0.3431 - participant_output_loss: 0.0591 - command_output_loss: 0.2840 - command_output_1_loss: 2.7039e-05 - participant_output_1_loss: 1.4893e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991

Epoch 9/10
18/18 [==============================] - 3s 186ms/step - loss: 0.7786 - participant_output_loss: 0.1022 - command_output_loss: 0.6763 - command_output_1_loss: 4.9820e-05 - participant_output_1_loss: 3.6505e-05 - participant_output_accuracy: 0.9973 - command_output_accuracy: 0.9964 - command_output_1_accuracy: 0.0518 - participant_output_1_accuracy: 0.1836 - val_loss: 0.8121 - val_participant_output_loss: 0.1430 - val_command_output_loss: 0.6690 - val_command_output_1_loss: 2.5444e-05 - val_participant_output_1_loss: 4.3319e-05 - val_participant_output_accuracy: 0.9797 - val_command_output_accuracy: 0.9834 - val_command_output_1_accuracy: 0.1492 - val_participant_output_1_accuracy: 0.1713
Epoch 10/10
18/18 [==============================] - 3s 144ms/step - loss: 0.6567 - participant_output_loss: 0.0849 - command_output_loss: 0.5717 - command_output_1_loss: 5.5279e-05 - participant_output_1_loss: 3.4831e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.995

...

Epoch 8/10
18/18 [==============================] - 3s 149ms/step - loss: 1.4687 - participant_output_loss: 0.1327 - command_output_loss: 1.3360 - command_output_1_loss: 5.0842e-05 - participant_output_1_loss: 2.2602e-05 - participant_output_accuracy: 0.9973 - command_output_accuracy: 0.9900 - command_output_1_accuracy: 0.1727 - participant_output_1_accuracy: 0.1845 - val_loss: 1.4856 - val_participant_output_loss: 0.1757 - val_command_output_loss: 1.3098 - val_command_output_1_loss: 4.2433e-05 - val_participant_output_1_loss: 2.5829e-05 - val_participant_output_accuracy: 0.9779 - val_command_output_accuracy: 0.9724 - val_command_output_1_accuracy: 0.5101 - val_participant_output_1_accuracy: 0.1068
Epoch 9/10
18/18 [==============================] - 3s 154ms/step - loss: 1.3324 - participant_output_loss: 0.1078 - command_output_loss: 1.2245 - command_output_1_loss: 3.9327e-05 - participant_output_1_loss: 2.1690e-05 - participant_output_accuracy: 0.9991 - command_output_accuracy: 0.9936

Epoch 10/10
18/18 [==============================] - 2s 122ms/step - loss: 0.3832 - participant_output_loss: 0.0281 - command_output_loss: 0.3551 - command_output_1_loss: 3.6749e-05 - participant_output_1_loss: 1.4217e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1745 - val_loss: 0.4861 - val_participant_output_loss: 0.0826 - val_command_output_loss: 0.4035 - val_command_output_1_loss: 3.3144e-05 - val_participant_output_1_loss: 2.1909e-05 - val_participant_output_accuracy: 0.9816 - val_command_output_accuracy: 0.9834 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1565
Epoch 1/10
18/18 [==============================] - 4s 240ms/step - loss: 3.3249 - participant_output_loss: 1.2605 - command_output_loss: 2.0585 - command_output_1_loss: 2.7397e-04 - participant_output_1_loss: 0.0055 - participant_output_accuracy: 0.5718 - command_output_accuracy: 0

Epoch 2/10
18/18 [==============================] - 3s 147ms/step - loss: 0.3646 - participant_output_loss: 0.0892 - command_output_loss: 0.2754 - command_output_1_loss: 2.7563e-05 - participant_output_1_loss: 3.9281e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9982 - command_output_1_accuracy: 0.2482 - participant_output_1_accuracy: 0.2236 - val_loss: 0.4510 - val_participant_output_loss: 0.1442 - val_command_output_loss: 0.3067 - val_command_output_1_loss: 3.4578e-05 - val_participant_output_1_loss: 5.3845e-05 - val_participant_output_accuracy: 0.9779 - val_command_output_accuracy: 0.9834 - val_command_output_1_accuracy: 0.0018 - val_participant_output_1_accuracy: 0.1786
Epoch 3/10
18/18 [==============================] - 2s 126ms/step - loss: 0.3062 - participant_output_loss: 0.0791 - command_output_loss: 0.2270 - command_output_1_loss: 2.6538e-05 - participant_output_1_loss: 3.6828e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9982

Epoch 4/10
18/18 [==============================] - 3s 177ms/step - loss: 1.5605 - participant_output_loss: 0.4090 - command_output_loss: 1.1514 - command_output_1_loss: 6.6435e-05 - participant_output_1_loss: 2.7615e-05 - participant_output_accuracy: 0.9664 - command_output_accuracy: 0.9573 - command_output_1_accuracy: 0.0100 - participant_output_1_accuracy: 0.2645 - val_loss: 1.4497 - val_participant_output_loss: 0.3873 - val_command_output_loss: 1.0624 - val_command_output_1_loss: 8.3884e-05 - val_participant_output_1_loss: 2.5922e-05 - val_participant_output_accuracy: 0.9595 - val_command_output_accuracy: 0.9503 - val_command_output_1_accuracy: 0.0037 - val_participant_output_1_accuracy: 0.0829
Epoch 5/10
18/18 [==============================] - 3s 170ms/step - loss: 1.2688 - participant_output_loss: 0.3159 - command_output_loss: 0.9527 - command_output_1_loss: 8.2811e-05 - participant_output_1_loss: 2.3579e-05 - participant_output_accuracy: 0.9818 - command_output_accuracy: 0.9673

Epoch 6/10
18/18 [==============================] - 2s 122ms/step - loss: 0.1399 - participant_output_loss: 0.0414 - command_output_loss: 0.0985 - command_output_1_loss: 1.2590e-05 - participant_output_1_loss: 1.4181e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991 - command_output_1_accuracy: 0.1045 - participant_output_1_accuracy: 0.1827 - val_loss: 0.2420 - val_participant_output_loss: 0.0941 - val_command_output_loss: 0.1479 - val_command_output_1_loss: 1.0417e-05 - val_participant_output_1_loss: 1.8180e-05 - val_participant_output_accuracy: 0.9797 - val_command_output_accuracy: 0.9834 - val_command_output_1_accuracy: 0.0700 - val_participant_output_1_accuracy: 0.2192
Epoch 7/10
18/18 [==============================] - 2s 124ms/step - loss: 0.1228 - participant_output_loss: 0.0378 - command_output_loss: 0.0850 - command_output_1_loss: 9.5328e-06 - participant_output_1_loss: 1.3296e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991

In [28]:
resnet_model = tf.keras.models.load_model('Initial_group_model_mix_20&s2_30_0608.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_c = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_s = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_c,acc_s)

0.9945 0.9816


In [29]:
Perfomance

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,10,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,10,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,10,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,10,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,10,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Group,20&s2_30,65.672576,0.9908,0.9834,0.973,0.9820,1.0000,0.9537,0.9913,0.9913,0.9908,0.9908,1.00,1.00,0.68,0.424,0.2970
86,Group,20&s2_30,62.740435,0.9926,0.9816,0.982,0.9640,1.0000,0.9722,0.9913,0.9913,0.9908,0.9817,1.00,1.00,0.66,0.424,0.3168
87,Group,20&s2_30,60.706642,0.9945,0.9779,0.991,0.9730,1.0000,0.9444,1.0000,0.9913,0.9817,0.9817,1.00,1.00,0.61,0.464,0.3069
88,Group,20&s2_30,60.410302,0.9926,0.9816,0.982,0.9820,1.0000,0.9444,0.9913,0.9913,0.9908,0.9908,1.00,1.00,0.68,0.480,0.3267


In [30]:
Perfomance.to_csv('Performance_0608_training_Data_Size.csv',index=False)

In [31]:
best_index

1